FCN: VGG16에서 FC layer에 해당하는 부분을 Conv layer로 만들어서 segmentation 성능 향상<br>
1. Conv로 만들어진 feature맵에 FC를 적용하면 원본 이미지에서의 좌표정보가 사라져서 segmentation 성능이 떨어지는데 이를 보완하기 위해 FC layer를 Conv화 한 모델<br>
2. FC layer를 없애고 Conv2DTranspose를 이용해 upsampling을 통한 decoding으로 segmentation 수행
3. upsampling 후 1x1 Conv를 왜 하는지는 아직 이해 못함 이 부분 더 생각 해봐야됨

원본 논문
- https://www.cv-foundation.org/openaccess/content_cvpr_2015/papers/Long_Fully_Convolutional_Networks_2015_CVPR_paper.pdf

### VGG16

In [ ]:
import tensorflow as tf
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, GlobalAveragePooling2D, Dense, Dropout
from keras.layers import BatchNormalization, Activation, Concatenate
from keras.models import Model

def VGG_Conv2D(z, n_convs=2, f=64, k=(3, 3), s=(1, 1), p="same"):
    """
    input
        : input tensor of Convolution layer
    n_convs
        : Number of iterations of Conv-Pooling
    f
        : filter size of Conv
    k
        : kernel size of Conv
    s
        : stride size of Conv
    p
        : padding status of Conv
    """

    for i in range(n_convs):
        z = Conv2D(filters=f, kernel_size=k, strides=s, padding=p, use_bias=False)(z)
        z = BatchNormalization()(z)
        z = Activation("relu")(z)
    z = MaxPooling2D(pool_size=(2, 2), strides=2)(z)
    return z

input = Input([224, 224, 3], dtype=tf.float32)

x = VGG_Conv2D(input, n_convs=2, f=64, k=(3, 3), s=(1, 1), p="same")
x = VGG_Conv2D(x, f=128)
x = VGG_Conv2D(x, n_convs=3, f=256)
x = VGG_Conv2D(x, n_convs=3, f=512)
x = VGG_Conv2D(x, n_convs=3, f=512)

x = Flatten()(x)
# x = GlobalAveragePooling2D()(x)

x = Dense(4096, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)

x = Dense(4096, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation("relu")(x)
x = Dropout(0.5)(x)

output = Dense(1000, activation="softmax")(x)

model = Model(inputs=input, outputs=output)

### FCN

In [ ]:
tf.keras.layers.UpSampling2D(
    size=(2, 2), data_format=None, interpolation='bilinear', **kwargs
)

In [ ]:
input = Input([224, 224, 3], dtype=tf.float32)

x = VGG_Conv2D(input, n_convs=2, f=64, k=(3, 3), s=(1, 1), p="same")
x = VGG_Conv2D(x, f=128)
x = VGG_Conv2D(x, n_convs=3, f=256)
f3 = x # (28, 28, 256)
x = VGG_Conv2D(x, n_convs=3, f=512)
f4 = x # (14, 14, 512)
x = VGG_Conv2D(x, n_convs=3, f=512)
f5 = x # (7, 7, 512)

Upsampling - Conv2DTranspose
- https://velog.io/@hayaseleu/Transposed-Convolutional-Layer%EC%9D%80-%EB%AC%B4%EC%97%87%EC%9D%B8%EA%B0%80

In [ ]:
# Use PASCAL VOC (including background)
n_classes = 21

#### FCN-32

In [ ]:
# FCN-32 output
# 32x Upsampling for 5th feature map and use predict result
fcn32 = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(32, 32), strides=(32, 32), use_bias=False)(f5)
fcn32 = tf.keras.layers.Activation('softmax')(fcn32)

print(fcn32.shape)

(None, 224, 224, 21)


#### FCN-16

In [ ]:
# 4x Upsampling for 5th feature map
x = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(4, 4), strides=(2, 2), use_bias=False)(f5) # (16, 16, n)
x = tf.keras.layers.Cropping2D(cropping=(1, 1))(x) # (14, 14, n)

# 1x1 Conv for 4th feature map
x2 = tf.keras.layers.Conv2D(n_classes, (1, 1), activation='relu', padding='same')(f4) # (14, 14, n)

# concat result of upsampling and 1x1 Conv
x = Concatenate()([x, x2]) # (14, 14, n)

# 16x Upsampling for x and use predict result
fcn16 = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(16, 16), strides=(16, 16), use_bias=False)(x)
fcn16 = tf.keras.layers.Activation('softmax')(fcn16)

print(fcn16.shape)

(None, 224, 224, 21)


#### FCN-8

In [ ]:
# 4x Upsampling for 5th feature map
x = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(4, 4), strides=(2, 2), use_bias=False)(f5) # (16, 16, n)
x = tf.keras.layers.Cropping2D(cropping=(1, 1))(x) # (14, 14, n)

# 1x1 Convolution for 4th feature map
x2 = tf.keras.layers.Conv2D(n_classes, (1, 1), activation='relu', padding='same')(f4) # (14, 14, n)

# concat result of upsampling and 1x1 Conv
x = Concatenate()([x, x2]) # (14, 14, n)

# 4x Upsampling for x
x = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(4, 4), strides=(2, 2), use_bias=False)(x) # (30, 30, n)
x = tf.keras.layers.Cropping2D(cropping=(1, 1))(x) # (28, 28, n)

# 1x1 Conn for 3rd feature map
x2 = tf.keras.layers.Conv2D(n_classes, (1, 1), activation='relu', padding='same')(f3) # (28, 28, n)

# concat result of upsampling and 1x1 Conv
x = Concatenate()([x, x2]) # (28, 28, n)

# 8x Upsampling for 5th feature map and use predict result
x = tf.keras.layers.Conv2DTranspose(n_classes, kernel_size=(8, 8), strides=(8, 8), use_bias=False)(x) # (224, 224, n)
fcn8 = tf.keras.layers.Activation('softmax')(x)

print(fcn8.shape)

(None, 224, 224, 21)
